# 17.0. Data Integration - File Analysis

In [7]:
import requests
import json

# Replace 'YOUR_API_KEY' with your actual Malware Bazaar API key
api_key = ''

# URL of the Malware Bazaar API
api_url = 'https://mb-api.abuse.ch/api/v1/'

# SHA-256 hash of the file
file_sha256 = 'eb0e81598d8526d88cac4695a3e9360cc8fbb331'

# Make a query to the Malware Bazaar API
response = requests.post(api_url, data={'query': 'get_info', 'hash': file_sha256,})

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    result = response.json()
    formatted_json = json.dumps(result, indent=2)

    print(formatted_json)
else:
    # Print an error message if the request was not successful
    print(f"Error: {response.status_code} - {response.text}")


{
  "query_status": "ok",
  "data": [
    {
      "sha256_hash": "e167b20f1acf48f7ce0ae33a218e2c1b300b41c012ededf03e7a3522a4ebe95e",
      "sha3_384_hash": "84f63fa87f733c8910e215ef5c96d90b97bfd17c86f5bac8580498198ec97494493addf29813ff9eaa1aed473ad51f6e",
      "sha1_hash": "eb0e81598d8526d88cac4695a3e9360cc8fbb331",
      "md5_hash": "7338b335ad5471cb67658f27836374f0",
      "first_seen": "2020-03-01 05:57:01",
      "last_seen": null,
      "file_name": "file",
      "file_size": 145408,
      "file_type_mime": "application/x-dosexec",
      "file_type": "exe",
      "reporter": "viql",
      "origin_country": null,
      "anonymous": 0,
      "signature": "RevengeRAT",
      "imphash": "f34d5f2d4577ed6d9ceec516c1f5a744",
      "tlsh": "D0E36A3DB74BCF12C96C06B800E6751467F067976B33DA0A6DCD60EE6A933421D0A6DE",
      "telfhash": null,
      "gimphash": null,
      "ssdeep": "1536:sGZWpdxayQAcj7gwluW14Z2II0ocL8ppVy8U+M76JMaDZVB0pI6lxYGinsPF9WPz:7dPIocwpPMGP9SBlxp+st9gTp5",
      "dhash_i

In [8]:
from datetime import datetime
import os
import time
import pandas as pd 
import requests
import urllib3
import json
import sys
import numpy as np

pd.options.display.max_colwidth = 1000
pd.set_option('display.max_columns', None)

In [9]:
path_OTX_Entries     = r"C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\OTX_Alien_Vault\\Entries.csv"
path_OTX_Anomalies   = r"C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\OTX_Alien_Vault\\Anomalies.csv"
path_OTX_Attacks     = r"C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\OTX_Alien_Vault\\Attacks.csv"
path_OTX_TV_Analysis = r"C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\OTX_Alien_Vault\\TV_Analysis.csv"
path_OTX_Behaviours  = r"C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\OTX_Alien_Vault\\Behaviours.csv"

In [10]:
path_MB_Entries     = r"C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\Malware_Bazaar\\Entries.csv"
path_MB_Tags        = r"C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\Malware_Bazaar\\Tags.csv"
path_MB_YaraRules   = r"C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\Malware_Bazaar\\YaraRules.csv"
path_MB_TV_Analysis = r"C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\Malware_Bazaar\\TV_Analysis.csv"
path_MB_Behaviours  = r"C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\Malware_Bazaar\\Behaviours.csv"

## Entries

In [33]:
entries_MB = pd.read_csv(path_MB_Entries, low_memory = False)
left_key = ['sha1', 'sha256', 'md5']
entries_MB.head(3)

,sha1,sha256,md5,first_seen,last_seen,file_type,delivery_method
0,411535c9548f86034a6a9a603de521a86a3b5466,04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d,5b5a0225a2b7b9a054417662c733168a,2021-11-09,2021-11-09,exe,email attachment
1,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbfb89147401c1e9e6e4f,da9534900ee0d11c9b30cf33152ea03c,2023-07-19,2023-08-25,exe,web download
2,8ab63abfe58fb218cc001581ac0d9fe38c784157,7f514ed5e1ec262953e6252a4089531c519e95d700c5808415b0f049fc59a5f0,855232a26e8afcd0cca5f816a1919dee,2022-01-04,2022-02-04,exe,unknown


In [34]:
entries_OTX = pd.read_csv(path_OTX_Entries, low_memory = False)
right_key = ['sha1', 'sha256', 'md5']
entries_OTX.head(3)

,sha1,sha256,md5,file_type,analysis_date
0,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692a664b28aa67dd0d100,d6cda532ce834b973d5292e05e1d6c38,exe,2022-02-25
1,713a00b13a87c52b43404cd32d0f4645844709d9,3b96a384f9989f28315eb374eae991e70ed07b88665cbf831f24bd574502feac,97ad0714b0245825407dd25d7d168d31,exe,2018-11-21
2,e900de5aea86567fadc7507a6811596eb4b86a14,3363270a2950c8518e7883ea147b3f947b3a96aa31d37a990c73f975946335ce,e076afd88fe6a708d91a3f771754e860,exe,2022-07-07


In [35]:
entries = pd.merge(entries_MB, entries_OTX, left_on=left_key, right_on=right_key, how='outer')
entries

,sha1,sha256,md5,first_seen,last_seen,file_type_x,delivery_method,file_type_y,analysis_date
0,411535c9548f86034a6a9a603de521a86a3b5466,04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d,5b5a0225a2b7b9a054417662c733168a,2021-11-09,2021-11-09,exe,email attachment,exe,2021-11-09
1,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbfb89147401c1e9e6e4f,da9534900ee0d11c9b30cf33152ea03c,2023-07-19,2023-08-25,exe,web download,exe,2023-07-19
2,8ab63abfe58fb218cc001581ac0d9fe38c784157,7f514ed5e1ec262953e6252a4089531c519e95d700c5808415b0f049fc59a5f0,855232a26e8afcd0cca5f816a1919dee,2022-01-04,2022-02-04,exe,unknown,exe,2022-01-04
3,55cbf8dae95d350ad7e5a63b418f6cd203488f51,8b7ae9f195b075a789d6d8277d500d27754bfa3c53ecca8db7beac8ccd07884f,97d9d216f2627cb238ca6637580b1739,2023-06-13,2023-06-13,exe,unknown,exe,2022-02-09
4,a2915c1be9e6134b7bf3ca5ca00eeb0c969bedab,6c2a2251861a6d2701814843fadac940cf4d34db9f446f0698352fd866b31739,1b09de36dfe5850d7e3fbd6b39c89a43,2022-03-27,2022-04-06,exe,web download,exe,2021-02-11
...,...,...,...,...,...,...,...,...,...
2657,6b6955af591154d3f6e2db35ca9904cce7e264b1,3a9d8c693b7dc6a9c2bf876c5626ebd0923e35bd5f698baed248df7f51f67275,03872a722df0b073a84fd1aa45eb9376,NaN,NaN,NaN,NaN,elf,2020-02-20
2658,3ab53037e2360c42b57d213234ced58717844a71,5de4932cb0ccab3fec711135cdd1af5a09c26a870bfb2d2adc650d0fdbf16488,e0fea510c784eb0e440e99e53d83d414,NaN,NaN,NaN,NaN,exe,2021-07-14
2659,4f9b9859a596a8bfdfee516788587ca0c0fb6ae8,f10dd5ce32d72489dd2696e5b58ee115f3cd065969a61a6613cab27bcd57de70,c9f771be253b6ca1d7a808d7d65c5e29,NaN,NaN,NaN,NaN,exe,2020-09-08
2660,d14540087dd6d350fb49e644e53ccefc0e4c1582,dba8a4717d516772b8cc06bd258265f21f7db6ddab62328917524c5a243c67a1,fe783ff4bccd76cb215ee1f7738d9ed5,NaN,NaN,NaN,NaN,exe,2021-12-20


#### Column Merging

- merge column 'file_type_x' and 'file_type_y'


In [36]:
# Verifica se, per tutte le righe, la colonna 'a' è uguale alla colonna 'b' oppure almeno una delle due è NaN
condition_for_all_rows = (entries.apply(lambda row: pd.isna(row['file_type_x']) or pd.isna(row['file_type_y']) or row['file_type_x'] == row['file_type_y'], axis=1)).all()

# Visualizza il risultato
print(f"La condizione è vera per tutte le righe: {condition_for_all_rows}")


La condizione è vera per tutte le righe: False


In [37]:
entries['file_type'] = entries['file_type_x'].combine_first(entries['file_type_y']).combine_first(entries['file_type_x'])

entries = entries.drop(columns=['file_type_x', 'file_type_y'])
entries

,sha1,sha256,md5,first_seen,last_seen,delivery_method,analysis_date,file_type
0,411535c9548f86034a6a9a603de521a86a3b5466,04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d,5b5a0225a2b7b9a054417662c733168a,2021-11-09,2021-11-09,email attachment,2021-11-09,exe
1,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbfb89147401c1e9e6e4f,da9534900ee0d11c9b30cf33152ea03c,2023-07-19,2023-08-25,web download,2023-07-19,exe
2,8ab63abfe58fb218cc001581ac0d9fe38c784157,7f514ed5e1ec262953e6252a4089531c519e95d700c5808415b0f049fc59a5f0,855232a26e8afcd0cca5f816a1919dee,2022-01-04,2022-02-04,unknown,2022-01-04,exe
3,55cbf8dae95d350ad7e5a63b418f6cd203488f51,8b7ae9f195b075a789d6d8277d500d27754bfa3c53ecca8db7beac8ccd07884f,97d9d216f2627cb238ca6637580b1739,2023-06-13,2023-06-13,unknown,2022-02-09,exe
4,a2915c1be9e6134b7bf3ca5ca00eeb0c969bedab,6c2a2251861a6d2701814843fadac940cf4d34db9f446f0698352fd866b31739,1b09de36dfe5850d7e3fbd6b39c89a43,2022-03-27,2022-04-06,web download,2021-02-11,exe
...,...,...,...,...,...,...,...,...
2657,6b6955af591154d3f6e2db35ca9904cce7e264b1,3a9d8c693b7dc6a9c2bf876c5626ebd0923e35bd5f698baed248df7f51f67275,03872a722df0b073a84fd1aa45eb9376,NaN,NaN,NaN,2020-02-20,elf
2658,3ab53037e2360c42b57d213234ced58717844a71,5de4932cb0ccab3fec711135cdd1af5a09c26a870bfb2d2adc650d0fdbf16488,e0fea510c784eb0e440e99e53d83d414,NaN,NaN,NaN,2021-07-14,exe
2659,4f9b9859a596a8bfdfee516788587ca0c0fb6ae8,f10dd5ce32d72489dd2696e5b58ee115f3cd065969a61a6613cab27bcd57de70,c9f771be253b6ca1d7a808d7d65c5e29,NaN,NaN,NaN,2020-09-08,exe
2660,d14540087dd6d350fb49e644e53ccefc0e4c1582,dba8a4717d516772b8cc06bd258265f21f7db6ddab62328917524c5a243c67a1,fe783ff4bccd76cb215ee1f7738d9ed5,NaN,NaN,NaN,2021-12-20,exe


In [38]:
ordered_entries_columns = ['sha1', 'sha256', 'md5', 'first_seen', 'last_seen', 'analysis_date', 'file_type', 'delivery_method' ]
entries = entries[ordered_entries_columns]
entries.head(3)

,sha1,sha256,md5,first_seen,last_seen,analysis_date,file_type,delivery_method
0,411535c9548f86034a6a9a603de521a86a3b5466,04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d,5b5a0225a2b7b9a054417662c733168a,2021-11-09,2021-11-09,2021-11-09,exe,email attachment
1,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbfb89147401c1e9e6e4f,da9534900ee0d11c9b30cf33152ea03c,2023-07-19,2023-08-25,2023-07-19,exe,web download
2,8ab63abfe58fb218cc001581ac0d9fe38c784157,7f514ed5e1ec262953e6252a4089531c519e95d700c5808415b0f049fc59a5f0,855232a26e8afcd0cca5f816a1919dee,2022-01-04,2022-02-04,2022-01-04,exe,unknown


## Threat Vendors Analysis

In [39]:
tv_analysis_MB = pd.read_csv(path_MB_TV_Analysis, low_memory = False)
tv_analysis_MB.head(3)

,sha1,sha256,md5,malware,verdict,threat_vendor_name
0,411535c9548f86034a6a9a603de521a86a3b5466,04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d,5b5a0225a2b7b9a054417662c733168a,pua.win.packer.borlanddelphi-15,NaN,clamav
1,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbfb89147401c1e9e6e4f,da9534900ee0d11c9b30cf33152ea03c,securiteinfo.com.trojan.generickd.68242337.20401.3632.unofficial,NaN,clamav
2,8ab63abfe58fb218cc001581ac0d9fe38c784157,7f514ed5e1ec262953e6252a4089531c519e95d700c5808415b0f049fc59a5f0,855232a26e8afcd0cca5f816a1919dee,win.trojan.remcos-9753190-0,NaN,clamav


In [40]:
tv_analysis_OTX = pd.read_csv(path_OTX_TV_Analysis, low_memory = False)
tv_analysis_OTX.head(3)

,sha1,sha256,md5,malware,verdict,threat_vendor_name
0,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692a664b28aa67dd0d100,d6cda532ce834b973d5292e05e1d6c38,win32:pwsx-gen\ [trj],malware infection,avast
1,713a00b13a87c52b43404cd32d0f4645844709d9,3b96a384f9989f28315eb374eae991e70ed07b88665cbf831f24bd574502feac,97ad0714b0245825407dd25d7d168d31,win32:malware-gen,malware infection,avast
2,e900de5aea86567fadc7507a6811596eb4b86a14,3363270a2950c8518e7883ea147b3f947b3a96aa31d37a990c73f975946335ce,e076afd88fe6a708d91a3f771754e860,win32:backdoorx-gen\ [trj],malware infection,avast


In [41]:
tv_analysis = pd.concat([tv_analysis_MB, tv_analysis_OTX])
tv_analysis.reset_index(drop=True, inplace=True)
tv_analysis

,sha1,sha256,md5,malware,verdict,threat_vendor_name
0,411535c9548f86034a6a9a603de521a86a3b5466,04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d,5b5a0225a2b7b9a054417662c733168a,pua.win.packer.borlanddelphi-15,NaN,clamav
1,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbfb89147401c1e9e6e4f,da9534900ee0d11c9b30cf33152ea03c,securiteinfo.com.trojan.generickd.68242337.20401.3632.unofficial,NaN,clamav
2,8ab63abfe58fb218cc001581ac0d9fe38c784157,7f514ed5e1ec262953e6252a4089531c519e95d700c5808415b0f049fc59a5f0,855232a26e8afcd0cca5f816a1919dee,win.trojan.remcos-9753190-0,NaN,clamav
3,8ab63abfe58fb218cc001581ac0d9fe38c784157,7f514ed5e1ec262953e6252a4089531c519e95d700c5808415b0f049fc59a5f0,855232a26e8afcd0cca5f816a1919dee,win.trojan.remcos-9841897-0,NaN,clamav
4,a2915c1be9e6134b7bf3ca5ca00eeb0c969bedab,6c2a2251861a6d2701814843fadac940cf4d34db9f446f0698352fd866b31739,1b09de36dfe5850d7e3fbd6b39c89a43,securiteinfo.com.generic.mg.1b09de36dfe5850d.24204.unofficial,NaN,clamav
...,...,...,...,...,...,...
9708,0d7b506cdc8e0228a8c50f1adf3b1b3224fc114d,a48ab00c2a748d146264d9ceebed2346013a730bde2ee91bc03de76f8d262aa1,1e53278bd1d1ee7c84ffc2d44dce4117,win.packed.zusy-6860439-0,malware infection,clamav
9709,a3447ba9b83f30284c6d3effb45c31ad9d5f258f,bbd3ecd9e9671d94e8897980c4eb9391ae9cb444615ed9a93b8221ae8fa66790,f6eec1317ece3ffb7c4916e224d9734d,win.trojan.revengerat-6344273-0,malware infection,clamav
9710,3ab53037e2360c42b57d213234ced58717844a71,5de4932cb0ccab3fec711135cdd1af5a09c26a870bfb2d2adc650d0fdbf16488,e0fea510c784eb0e440e99e53d83d414,win.malware.generic-6623004-0,malware infection,clamav
9711,4f9b9859a596a8bfdfee516788587ca0c0fb6ae8,f10dd5ce32d72489dd2696e5b58ee115f3cd065969a61a6613cab27bcd57de70,c9f771be253b6ca1d7a808d7d65c5e29,win.malware.dlmc-7395626-0,malware infection,clamav


## Behaviours

In [47]:
behaviours_MB = pd.read_csv(path_MB_Behaviours, low_memory = False)
behaviours_MB.rename(columns= { 'action_maliciousness_level' : 'maliciousness_level' }, inplace=True)

behaviours_MB.head(3)

,sha1,sha256,md5,action_name,maliciousness_level
0,8ab63abfe58fb218cc001581ac0d9fe38c784157,7f514ed5e1ec262953e6252a4089531c519e95d700c5808415b0f049fc59a5f0,855232a26e8afcd0cca5f816a1919dee,searching for the window,neutral
1,8ab63abfe58fb218cc001581ac0d9fe38c784157,7f514ed5e1ec262953e6252a4089531c519e95d700c5808415b0f049fc59a5f0,855232a26e8afcd0cca5f816a1919dee,сreating synchronization primitives,neutral
2,8ab63abfe58fb218cc001581ac0d9fe38c784157,7f514ed5e1ec262953e6252a4089531c519e95d700c5808415b0f049fc59a5f0,855232a26e8afcd0cca5f816a1919dee,unauthorized injection to a recently created process,neutral


In [48]:
behaviours_OTX = pd.read_csv(path_OTX_Behaviours, low_memory = False)
behaviours_OTX.rename(columns= { 'action_level' : 'maliciousness_level' }, inplace=True)

behaviours_OTX.head(3)

,sha1,sha256,md5,action_name,action_description,maliciousness_level
0,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692a664b28aa67dd0d100,d6cda532ce834b973d5292e05e1d6c38,checks debugger,checks if process is being debugged by a debugger,neutral
1,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692a664b28aa67dd0d100,d6cda532ce834b973d5292e05e1d6c38,generates crypto key,uses windows apis to generate a cryptographic key,neutral
2,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692a664b28aa67dd0d100,d6cda532ce834b973d5292e05e1d6c38,antivm memory available,"checks amount of memory in system, this can be used to detect virtual machines that have a low amount of memory available",neutral


In [49]:
behaviours = pd.concat([behaviours_OTX, behaviours_MB])
behaviours

,sha1,sha256,md5,action_name,action_description,maliciousness_level
0,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692a664b28aa67dd0d100,d6cda532ce834b973d5292e05e1d6c38,checks debugger,checks if process is being debugged by a debugger,neutral
1,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692a664b28aa67dd0d100,d6cda532ce834b973d5292e05e1d6c38,generates crypto key,uses windows apis to generate a cryptographic key,neutral
2,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692a664b28aa67dd0d100,d6cda532ce834b973d5292e05e1d6c38,antivm memory available,"checks amount of memory in system, this can be used to detect virtual machines that have a low amount of memory available",neutral
3,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692a664b28aa67dd0d100,d6cda532ce834b973d5292e05e1d6c38,allocates rwx,allocates read-write-execute memory (usually to unpack itself),neutral
4,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692a664b28aa67dd0d100,d6cda532ce834b973d5292e05e1d6c38,antivm network adapters,checks adapter addresses which can be used to detect virtual network interfaces,neutral
...,...,...,...,...,...,...
9547,5d0cf2954a43c613f270ee9c6ea2e74798787725,eaf087b8fe0cf69bed568070ae460d63483525842d4c84c7750ef87c4985e622,7b25379caa4340a714b210dfd01fd5f9,asyncrat,NaN,malicious
9548,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9636a8c8be52858347d,aff1636c687b809a106aadf5eb0e40ee,contacts a large (345195) amount of remote hosts,NaN,malicious
9549,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9636a8c8be52858347d,aff1636c687b809a106aadf5eb0e40ee,creates a large amount of network flows,NaN,malicious
9550,4b3f7c7b3d8f6748505115fc291b6ac02e1b99a0,c3217157c0ab2d83047f679a073d19d296a03306072ef9636a8c8be52858347d,aff1636c687b809a106aadf5eb0e40ee,modifies the watchdog daemon,NaN,malicious


## Yara Rules

In [50]:
yara_rules = pd.read_csv(path_MB_YaraRules, low_memory = False)
yara_rules.head(3)

,sha1,sha256,md5,yara_rule_name,yara_rule_description,yara_rule_author
0,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbfb89147401c1e9e6e4f,da9534900ee0d11c9b30cf33152ea03c,iexplorer remcos,detect iexplorer being taken over by remcos,iam-py-test
1,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbfb89147401c1e9e6e4f,da9534900ee0d11c9b30cf33152ea03c,indicator suspicious exe uacbypass cmstpcom,detects windows exceutables bypassing uac using cmstp com interfaces. mitre (t1218.003),ditekSHen
2,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbfb89147401c1e9e6e4f,da9534900ee0d11c9b30cf33152ea03c,pe imphash,NaN,NaN


## Anomalies

In [51]:
anomalies = pd.read_csv(path_OTX_Anomalies, low_memory = False)
anomalies

,sha1,sha256,md5,anomaly_name
0,b701b181de8e595225668cb65d5b975e5b300a39,58edde581177c6f96caa4da4cd73396afee160b9152692a664b28aa67dd0d100,d6cda532ce834b973d5292e05e1d6c38,checksum header zero
1,e900de5aea86567fadc7507a6811596eb4b86a14,3363270a2950c8518e7883ea147b3f947b3a96aa31d37a990c73f975946335ce,e076afd88fe6a708d91a3f771754e860,checksum header zero
2,12d263832d7f08122fbfc4431da22016e0823539,cf1260b1d0686dbee4a5c9e3bcd09eff07bfae0a53d17356b72150f59ad1a497,497d3c4d08b80e03f622064dced10135,entropy based
3,12d263832d7f08122fbfc4431da22016e0823539,cf1260b1d0686dbee4a5c9e3bcd09eff07bfae0a53d17356b72150f59ad1a497,497d3c4d08b80e03f622064dced10135,checksum header zero
4,18a703514e47e1c3b59cf12de95f1ea5c7475083,28b6bbc6b1d6ee21ccdc25c5eefccad5e21556954542cd19a0a87f7dde91df78,260bf0106c006d4e988a2685df71a758,checksum header zero
...,...,...,...,...
3233,4f9b9859a596a8bfdfee516788587ca0c0fb6ae8,f10dd5ce32d72489dd2696e5b58ee115f3cd065969a61a6613cab27bcd57de70,c9f771be253b6ca1d7a808d7d65c5e29,entropy based
3234,4f9b9859a596a8bfdfee516788587ca0c0fb6ae8,f10dd5ce32d72489dd2696e5b58ee115f3cd065969a61a6613cab27bcd57de70,c9f771be253b6ca1d7a808d7d65c5e29,checksum header zero
3235,ba5e50a39d1502e38196724ec9d844ee4bf8d002,f3fcbb0fedb1e3b732185aebbf845ca185c950ca3635026d8a754312220577c9,0550342e66698384f232ac39b700dd14,checksum header zero
3236,d14540087dd6d350fb49e644e53ccefc0e4c1582,dba8a4717d516772b8cc06bd258265f21f7db6ddab62328917524c5a243c67a1,fe783ff4bccd76cb215ee1f7738d9ed5,checksum header zero


## Attacks

In [52]:
attacks = pd.read_csv(path_OTX_Attacks, low_memory = False)
attacks

,sha1,sha256,md5,attack_id,attack_name,attack_type
0,713a00b13a87c52b43404cd32d0f4645844709d9,3b96a384f9989f28315eb374eae991e70ed07b88665cbf831f24bd574502feac,97ad0714b0245825407dd25d7d168d31,T1060,Registry Run Keys / Start Folder,Installation
1,713a00b13a87c52b43404cd32d0f4645844709d9,3b96a384f9989f28315eb374eae991e70ed07b88665cbf831f24bd574502feac,97ad0714b0245825407dd25d7d168d31,T1056,Input Capture,Objectives
2,e900de5aea86567fadc7507a6811596eb4b86a14,3363270a2950c8518e7883ea147b3f947b3a96aa31d37a990c73f975946335ce,e076afd88fe6a708d91a3f771754e860,T1060,Registry Run Keys / Start Folder,Installation
3,12d263832d7f08122fbfc4431da22016e0823539,cf1260b1d0686dbee4a5c9e3bcd09eff07bfae0a53d17356b72150f59ad1a497,497d3c4d08b80e03f622064dced10135,T1119,Automated Collection,Objectives
4,12d263832d7f08122fbfc4431da22016e0823539,cf1260b1d0686dbee4a5c9e3bcd09eff07bfae0a53d17356b72150f59ad1a497,497d3c4d08b80e03f622064dced10135,T1045,Software Packing,Installation
...,...,...,...,...,...,...
5391,d14540087dd6d350fb49e644e53ccefc0e4c1582,dba8a4717d516772b8cc06bd258265f21f7db6ddab62328917524c5a243c67a1,fe783ff4bccd76cb215ee1f7738d9ed5,T1055,"Defense Evasion, Privilege Escalation",Installation
5392,98837feaac2ed17b3709e5183b660dd41e7a1e0e,5b37ddd18800dc91071db31b340e8f9997bb89d42d5a194d7ff8c980a54abe89,1b47215513edd622ba825dced07c8969,T1060,Registry Run Keys / Start Folder,Installation
5393,98837feaac2ed17b3709e5183b660dd41e7a1e0e,5b37ddd18800dc91071db31b340e8f9997bb89d42d5a194d7ff8c980a54abe89,1b47215513edd622ba825dced07c8969,T1045,Software Packing,Installation
5394,98837feaac2ed17b3709e5183b660dd41e7a1e0e,5b37ddd18800dc91071db31b340e8f9997bb89d42d5a194d7ff8c980a54abe89,1b47215513edd622ba825dced07c8969,T1055,"Defense Evasion, Privilege Escalation",Installation


## Tags

In [53]:
tags = pd.read_csv(path_MB_Tags, low_memory = False)
tags.head(3)

,sha1,sha256,md5,tag_name
0,411535c9548f86034a6a9a603de521a86a3b5466,04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d,5b5a0225a2b7b9a054417662c733168a,exe
1,411535c9548f86034a6a9a603de521a86a3b5466,04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d,5b5a0225a2b7b9a054417662c733168a,remcosrat
2,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbfb89147401c1e9e6e4f,da9534900ee0d11c9b30cf33152ea03c,32


# Save dataset

In [54]:
dataframes_dict = {}

# Add the DataFrames to the dictionary with keys
dataframes_dict['Entries'] = entries
dataframes_dict['TV_Analysis'] = tv_analysis
dataframes_dict['Yara_Rules'] = yara_rules
dataframes_dict['Tags'] = tags
dataframes_dict['Behaviours'] = behaviours
dataframes_dict['Anomalies'] = anomalies
dataframes_dict['Attacks'] = attacks

In [55]:
save_directory = "C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataIntegration\\"
for key, df in dataframes_dict.items():
    file_name = f"{key}.csv"
    file_path = save_directory + file_name
    df.to_csv(file_path, index=False)
    print(f'{key} added')

Entries added
TV_Analysis added
Yara_Rules added
Tags added
Behaviours added
Anomalies added
Attacks added
